In [ ]:
import sys
import warnings

import numpy
import numpy as np
import math
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn import metrics
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt

import time

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import os

# Load your original dataset
df = pd.read_excel(r"UP08.xlsx", index_col='Date_Time')
df = df[['dissolvedoxygenmeasured']]
df=df['2017-04-01':'2021-03-31']

In [ ]:
# Cluster Datasets:
# Cluster 1: November, December, January, February
# Cluster 2: June, July, August
# Cluster 3: April, May, September, October, March

### Cluster 1

In [ ]:
# Cluster Datasets:
# Cluster 1: November, December, January, February

In [ ]:
# Load your original dataset
df = pd.read_excel(r"UP08.xlsx", index_col='Date_Time')
df = df[['dissolvedoxygenmeasured']]
df=df['2017-04-01':'2021-03-31']

# Filter DataFrame for the months November, December, January, February
df = df[df.index.month.isin([11, 12, 1, 2])]

# Print or use the resulting DataFrame
print(df)

In [ ]:
df_train=df['2017-04-01':'2020-03-31']
df_test=df['2020-04-01':]
df_train.shape, df_test.shape

In [ ]:
# convert an array of values into a dataset matrix
import numpy 
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-2):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back+2, 0])
    return numpy.array(dataX), numpy.array(dataY)

def percentage_error(actual, predicted):
    res = numpy.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred): 
    return numpy.mean(numpy.abs(percentage_error(numpy.asarray(y_true), numpy.asarray(y_pred)))) * 100


In [ ]:
datasets=df.values
X, Y = create_dataset(datasets, 5)

train_size = 8664
test_size = 2880
trainX, testX = X[0:train_size], X[train_size:]
trainY, testY = Y[0:train_size], Y[train_size:]

In [ ]:
#Linear Regression
start_time = time.time()

numpy.random.seed(1234)
tf.random.set_seed(1234)

X_train=pd.DataFrame(trainX)
Y_train=pd.DataFrame(trainY)
X_test=pd.DataFrame(testX)
Y_test=pd.DataFrame(testY)

sc_X = MinMaxScaler()
sc_y = MinMaxScaler()

X= sc_X.fit_transform(X_train)
y= sc_y.fit_transform(Y_train)
X1= sc_X.transform(X_test)
y1= sc_y.transform(Y_test)
y=y.ravel()
y1=y1.ravel()  


from sklearn.linear_model import LinearRegression

lr = LinearRegression()

lr.fit(X,y)
y_pred_train_lr= lr.predict(X)
y_pred_test_lr= lr.predict(X1)

y_pred_train_lr=pd.DataFrame(y_pred_train_lr)
y_pred_test_lr=pd.DataFrame(y_pred_test_lr)

y1=pd.DataFrame(y1)
y=pd.DataFrame(y)

y_pred_test1_lr= sc_y.inverse_transform (y_pred_test_lr)
y_pred_train1_lr=sc_y.inverse_transform (y_pred_train_lr)

y_test= sc_y.inverse_transform (y1)
y_train= sc_y.inverse_transform (y)

y_pred_test1_rf=pd.DataFrame(y_pred_test1_lr)
y_pred_train1_rf=pd.DataFrame(y_pred_train1_lr)

y_test= pd.DataFrame(y_test)


#summarize the fit of the model on train data
mape_train_lr_c1 = mean_absolute_percentage_error(y_train,y_pred_train1_lr)
rmse_train_lr_c1 = sqrt(mean_squared_error(y_train,y_pred_train1_lr))
mae_train_lr_c1 = metrics.mean_absolute_error(y_train,y_pred_train1_lr)

# summarize the fit of the model on test data
mape_test_lr_c1 = mean_absolute_percentage_error(y_test,y_pred_test1_lr)
rmse_test_lr_c1 = sqrt(mean_squared_error(y_test,y_pred_test1_lr))
mae_test_lr_c1 = metrics.mean_absolute_error(y_test,y_pred_test1_lr)

# train scores
print("The metrics for the training data are: ")
print(mape_train_lr_c1)
print(rmse_train_lr_c1)
print(mae_train_lr_c1)

# test scores
print("The metrics for the testing data are: ")
print(mape_test_lr_c1)
print(rmse_test_lr_c1)
print(mae_test_lr_c1)

lr_time=time.time() - start_time
print("--- %s seconds - Linear Regression ---" % (lr_time))


In [ ]:
# Support Vector Regression
start_time = time.time()

numpy.random.seed(1234)
tf.random.set_seed(1234)

X_train=pd.DataFrame(trainX)
Y_train=pd.DataFrame(trainY)
X_test=pd.DataFrame(testX)
Y_test=pd.DataFrame(testY)

sc_X = StandardScaler()
sc_y = StandardScaler()

X= sc_X.fit_transform(X_train)
y= sc_y.fit_transform(Y_train)
X1= sc_X.transform(X_test)
y1= sc_y.transform(Y_test)
y=y.ravel()
y1=y1.ravel()  

from sklearn.svm import SVR

svr = SVR()

svr.fit(X,y)
y_pred_train_svr= svr.predict(X)
y_pred_test_svr= svr.predict(X1)

y_pred_train_svr=pd.DataFrame(y_pred_train_svr)
y_pred_test_svr=pd.DataFrame(y_pred_test_svr)

y1=pd.DataFrame(y1)
y=pd.DataFrame(y)

y_pred_test1_svr= sc_y.inverse_transform (y_pred_test_svr)
y_pred_train1_svr=sc_y.inverse_transform (y_pred_train_svr)

y_test= sc_y.inverse_transform (y1)
y_train= sc_y.inverse_transform (y)

y_pred_test1_rf=pd.DataFrame(y_pred_test1_svr)
y_pred_train1_rf=pd.DataFrame(y_pred_train1_svr)

y_test= pd.DataFrame(y_test)


#summarize the fit of the model on train data
mape_train_svr_c1 = mean_absolute_percentage_error(y_train,y_pred_train1_svr)
rmse_train_svr_c1 = sqrt(mean_squared_error(y_train,y_pred_train1_svr))
mae_train_svr_c1 = metrics.mean_absolute_error(y_train,y_pred_train1_svr)

# summarize the fit of the model on test data
mape_test_svr_c1 = mean_absolute_percentage_error(y_test,y_pred_test1_svr)
rmse_test_svr_c1 = sqrt(mean_squared_error(y_test,y_pred_test1_svr))
mae_test_svr_c1 = metrics.mean_absolute_error(y_test,y_pred_test1_svr)

# train scores
print("The metrics for the training data are: ")
print(mape_train_svr_c1)
print(rmse_train_svr_c1)
print(mae_train_svr_c1)

# test scores
print("The metrics for the testing data are: ")
print(mape_test_svr_c1)
print(rmse_test_svr_c1)
print(mae_test_svr_c1)

svr_time=time.time() - start_time
print("--- %s seconds - Support Vector Regression ---" % (svr_time))


### Cluster 2

In [ ]:
# Cluster 2: June, July, August

In [ ]:
# Load your original dataset
df = pd.read_excel(r"UP08.xlsx", index_col='Date_Time')
df = df[['dissolvedoxygenmeasured']]
df=df['2017-04-01':'2021-03-31']

# Assuming you have already loaded and filtered your DataFrame df
# If not, you can use the provided code to load and filter it

# Filter DataFrame for the months 'June', 'July', 'August'
df = df[df.index.month.isin([6, 7, 8])]

# Print or use the resulting DataFrame
print(df)

In [ ]:
df_train=df['2017-04-01':'2020-03-31']
df_test=df['2020-04-01':]
df_train.shape, df_test.shape

In [ ]:
# convert an array of values into a dataset matrix
import numpy 
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-2):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back+2, 0])
    return numpy.array(dataX), numpy.array(dataY)

def percentage_error(actual, predicted):
    res = numpy.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred): 
    return numpy.mean(numpy.abs(percentage_error(numpy.asarray(y_true), numpy.asarray(y_pred)))) * 100


In [ ]:
datasets=df.values
X, Y = create_dataset(datasets, 5)

train_size = 6624
test_size = 2208
trainX, testX = X[0:train_size], X[train_size:]
trainY, testY = Y[0:train_size], Y[train_size:]

In [ ]:
#Linear Regression
start_time = time.time()

numpy.random.seed(1234)
tf.random.set_seed(1234)

X_train=pd.DataFrame(trainX)
Y_train=pd.DataFrame(trainY)
X_test=pd.DataFrame(testX)
Y_test=pd.DataFrame(testY)

sc_X = MinMaxScaler()
sc_y = MinMaxScaler()

X= sc_X.fit_transform(X_train)
y= sc_y.fit_transform(Y_train)
X1= sc_X.transform(X_test)
y1= sc_y.transform(Y_test)
y=y.ravel()
y1=y1.ravel()  


from sklearn.linear_model import LinearRegression

lr = LinearRegression()

lr.fit(X,y)
y_pred_train_lr= lr.predict(X)
y_pred_test_lr= lr.predict(X1)

y_pred_train_lr=pd.DataFrame(y_pred_train_lr)
y_pred_test_lr=pd.DataFrame(y_pred_test_lr)

y1=pd.DataFrame(y1)
y=pd.DataFrame(y)

y_pred_test1_lr= sc_y.inverse_transform (y_pred_test_lr)
y_pred_train1_lr=sc_y.inverse_transform (y_pred_train_lr)

y_test= sc_y.inverse_transform (y1)
y_train= sc_y.inverse_transform (y)

y_pred_test1_rf=pd.DataFrame(y_pred_test1_lr)
y_pred_train1_rf=pd.DataFrame(y_pred_train1_lr)

y_test= pd.DataFrame(y_test)


#summarize the fit of the model on train data
mape_train_lr_c2 = mean_absolute_percentage_error(y_train,y_pred_train1_lr)
rmse_train_lr_c2 = sqrt(mean_squared_error(y_train,y_pred_train1_lr))
mae_train_lr_c2 = metrics.mean_absolute_error(y_train,y_pred_train1_lr)

# summarize the fit of the model on test data
mape_test_lr_c2 = mean_absolute_percentage_error(y_test,y_pred_test1_lr)
rmse_test_lr_c2 = sqrt(mean_squared_error(y_test,y_pred_test1_lr))
mae_test_lr_c2 = metrics.mean_absolute_error(y_test,y_pred_test1_lr)

# train scores
print("The metrics for the training data are: ")
print(mape_train_lr_c2)
print(rmse_train_lr_c2)
print(mae_train_lr_c2)

# test scores
print("The metrics for the testing data are: ")
print(mape_test_lr_c2)
print(rmse_test_lr_c2)
print(mae_test_lr_c2)
lr_time=time.time() - start_time
print("--- %s seconds - Linear Regression ---" % (lr_time))

In [ ]:
# Support Vector Regression
start_time = time.time()

numpy.random.seed(1234)
tf.random.set_seed(1234)

X_train=pd.DataFrame(trainX)
Y_train=pd.DataFrame(trainY)
X_test=pd.DataFrame(testX)
Y_test=pd.DataFrame(testY)

sc_X = StandardScaler()
sc_y = StandardScaler()

X= sc_X.fit_transform(X_train)
y= sc_y.fit_transform(Y_train)
X1= sc_X.transform(X_test)
y1= sc_y.transform(Y_test)
y=y.ravel()
y1=y1.ravel()  

from sklearn.svm import SVR

svr = SVR()

svr.fit(X,y)
y_pred_train_svr= svr.predict(X)
y_pred_test_svr= svr.predict(X1)

y_pred_train_svr=pd.DataFrame(y_pred_train_svr)
y_pred_test_svr=pd.DataFrame(y_pred_test_svr)

y1=pd.DataFrame(y1)
y=pd.DataFrame(y)

y_pred_test1_svr= sc_y.inverse_transform (y_pred_test_svr)
y_pred_train1_svr=sc_y.inverse_transform (y_pred_train_svr)

y_test= sc_y.inverse_transform (y1)
y_train= sc_y.inverse_transform (y)

y_pred_test1_rf=pd.DataFrame(y_pred_test1_svr)
y_pred_train1_rf=pd.DataFrame(y_pred_train1_svr)

y_test= pd.DataFrame(y_test)


#summarize the fit of the model on train data
mape_train_svr_c2 = mean_absolute_percentage_error(y_train,y_pred_train1_svr)
rmse_train_svr_c2 = sqrt(mean_squared_error(y_train,y_pred_train1_svr))
mae_train_svr_c2 = metrics.mean_absolute_error(y_train,y_pred_train1_svr)

# summarize the fit of the model on test data
mape_test_svr_c2 = mean_absolute_percentage_error(y_test,y_pred_test1_svr)
rmse_test_svr_c2 = sqrt(mean_squared_error(y_test,y_pred_test1_svr))
mae_test_svr_c2 = metrics.mean_absolute_error(y_test,y_pred_test1_svr)

# train scores
print("The metrics for the training data are: ")
print(mape_train_svr_c2)
print(rmse_train_svr_c2)
print(mae_train_svr_c2)

# test scores
print("The metrics for the testing data are: ")
print(mape_test_svr_c2)
print(rmse_test_svr_c2)
print(mae_test_svr_c2)

svr_time=time.time() - start_time
print("--- %s seconds - Support Vector Regression ---" % (svr_time))


### Cluster 3

In [ ]:
# April 2017, May 2017, September 2017, October 2017, March 2018
# April 2018, May 2018, September 2018, October 2018, March 2019
# April 2019, May 2019, September 2019, October 2019, March 2020
# April 2020, May 2020, September 2020, October 2020, March 2021

In [ ]:
# Load your original dataset
df = pd.read_excel(r"UP08.xlsx", index_col='Date_Time')
df = df[['dissolvedoxygenmeasured']]
df=df['2017-04-01':'2021-03-31']


# Filter DataFrame for the months April, May, September, October, March

df = df[df.index.month.isin([3, 4, 5, 9, 10])]

# Print or use the resulting DataFrame
print(df)

In [ ]:
df_train=df['2017-04-01':'2020-03-31']
df_test=df['2020-04-01':]
df_train.shape, df_test.shape

In [ ]:
# convert an array of values into a dataset matrix
import numpy 
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-2):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back+2, 0])
    return numpy.array(dataX), numpy.array(dataY)

def percentage_error(actual, predicted):
    res = numpy.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred): 
    return numpy.mean(numpy.abs(percentage_error(numpy.asarray(y_true), numpy.asarray(y_pred)))) * 100


In [ ]:
datasets=df.values
X, Y = create_dataset(datasets, 5)

train_size = 11016
test_size = 3672
trainX, testX = X[0:train_size], X[train_size:]
trainY, testY = Y[0:train_size], Y[train_size:]

In [ ]:
#Linear Regression
start_time = time.time()

numpy.random.seed(1234)
tf.random.set_seed(1234)

X_train=pd.DataFrame(trainX)
Y_train=pd.DataFrame(trainY)
X_test=pd.DataFrame(testX)
Y_test=pd.DataFrame(testY)

sc_X = MinMaxScaler()
sc_y = MinMaxScaler()

X= sc_X.fit_transform(X_train)
y= sc_y.fit_transform(Y_train)
X1= sc_X.transform(X_test)
y1= sc_y.transform(Y_test)
y=y.ravel()
y1=y1.ravel()  


from sklearn.linear_model import LinearRegression

lr = LinearRegression()

lr.fit(X,y)
y_pred_train_lr= lr.predict(X)
y_pred_test_lr= lr.predict(X1)

y_pred_train_lr=pd.DataFrame(y_pred_train_lr)
y_pred_test_lr=pd.DataFrame(y_pred_test_lr)

y1=pd.DataFrame(y1)
y=pd.DataFrame(y)

y_pred_test1_lr= sc_y.inverse_transform (y_pred_test_lr)
y_pred_train1_lr=sc_y.inverse_transform (y_pred_train_lr)

y_test= sc_y.inverse_transform (y1)
y_train= sc_y.inverse_transform (y)

y_pred_test1_rf=pd.DataFrame(y_pred_test1_lr)
y_pred_train1_rf=pd.DataFrame(y_pred_train1_lr)

y_test= pd.DataFrame(y_test)


#summarize the fit of the model on train data
mape_train_lr_c3 = mean_absolute_percentage_error(y_train,y_pred_train1_lr)
rmse_train_lr_c3 = sqrt(mean_squared_error(y_train,y_pred_train1_lr))
mae_train_lr_c3 = metrics.mean_absolute_error(y_train,y_pred_train1_lr)

# summarize the fit of the model on test data
mape_test_lr_c3 = mean_absolute_percentage_error(y_test,y_pred_test1_lr)
rmse_test_lr_c3 = sqrt(mean_squared_error(y_test,y_pred_test1_lr))
mae_test_lr_c3 = metrics.mean_absolute_error(y_test,y_pred_test1_lr)

# train scores
print("The metrics for the training data are: ")
print(mape_train_lr_c3)
print(rmse_train_lr_c3)
print(mae_train_lr_c3)

# test scores
print("The metrics for the testing data are: ")
print(mape_test_lr_c3)
print(rmse_test_lr_c3)
print(mae_test_lr_c3)

lr_time=time.time() - start_time
print("--- %s seconds - Linear Regression ---" % (lr_time))


In [ ]:
# Support Vector Regression
start_time = time.time()

numpy.random.seed(1234)
tf.random.set_seed(1234)

X_train=pd.DataFrame(trainX)
Y_train=pd.DataFrame(trainY)
X_test=pd.DataFrame(testX)
Y_test=pd.DataFrame(testY)

sc_X = StandardScaler()
sc_y = StandardScaler()

X= sc_X.fit_transform(X_train)
y= sc_y.fit_transform(Y_train)
X1= sc_X.transform(X_test)
y1= sc_y.transform(Y_test)
y=y.ravel()
y1=y1.ravel()  

from sklearn.svm import SVR

svr = SVR()

svr.fit(X,y)
y_pred_train_svr= svr.predict(X)
y_pred_test_svr= svr.predict(X1)

y_pred_train_svr=pd.DataFrame(y_pred_train_svr)
y_pred_test_svr=pd.DataFrame(y_pred_test_svr)

y1=pd.DataFrame(y1)
y=pd.DataFrame(y)

y_pred_test1_svr= sc_y.inverse_transform (y_pred_test_svr)
y_pred_train1_svr=sc_y.inverse_transform (y_pred_train_svr)

y_test= sc_y.inverse_transform (y1)
y_train= sc_y.inverse_transform (y)

y_pred_test1_rf=pd.DataFrame(y_pred_test1_svr)
y_pred_train1_rf=pd.DataFrame(y_pred_train1_svr)

y_test= pd.DataFrame(y_test)


#summarize the fit of the model on train data
mape_train_svr_c3 = mean_absolute_percentage_error(y_train,y_pred_train1_svr)
rmse_train_svr_c3 = sqrt(mean_squared_error(y_train,y_pred_train1_svr))
mae_train_svr_c3 = metrics.mean_absolute_error(y_train,y_pred_train1_svr)

# summarize the fit of the model on test data
mape_test_svr_c3 = mean_absolute_percentage_error(y_test,y_pred_test1_svr)
rmse_test_svr_c3 = sqrt(mean_squared_error(y_test,y_pred_test1_svr))
mae_test_svr_c3 = metrics.mean_absolute_error(y_test,y_pred_test1_svr)

# train scores
print("The metrics for the training data are: ")
print(mape_train_svr_c3)
print(rmse_train_svr_c3)
print(mae_train_svr_c3)

# test scores
print("The metrics for the testing data are: ")
print(mape_test_svr_c3)
print(rmse_test_svr_c3)
print(mae_test_svr_c3)

svr_time=time.time() - start_time
print("--- %s seconds - Support Vector Regression ---" % (svr_time))


# whole

In [ ]:
# Load your original dataset
df = pd.read_excel(r"UP08.xlsx", index_col='Date_Time')
df = df[['dissolvedoxygenmeasured']]
df=df['2017-04-01':'2021-03-31']

df

In [ ]:
# convert an array of values into a dataset matrix
import numpy 
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-2):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back+2, 0])
    return numpy.array(dataX), numpy.array(dataY)

def percentage_error(actual, predicted):
    res = numpy.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred): 
    return numpy.mean(numpy.abs(percentage_error(numpy.asarray(y_true), numpy.asarray(y_pred)))) * 100


In [ ]:
# Split the data into training and testing sets
train_data = df.loc['2017-04-01':'2020-03-31']
test_data = df.loc['2020-04-01':]

In [ ]:
train_data.shape, test_data.shape

In [ ]:
datasets=df.values
X, Y = create_dataset(datasets, 5)

train_size = 26304
test_size = 8760
trainX, testX = X[0:train_size], X[train_size:]
trainY, testY = Y[0:train_size], Y[train_size:]

In [ ]:
#Linear Regression
start_time = time.time()

numpy.random.seed(1234)
tf.random.set_seed(1234)

X_train=pd.DataFrame(trainX)
Y_train=pd.DataFrame(trainY)
X_test=pd.DataFrame(testX)
Y_test=pd.DataFrame(testY)

sc_X = MinMaxScaler()
sc_y = MinMaxScaler()

X= sc_X.fit_transform(X_train)
y= sc_y.fit_transform(Y_train)
X1= sc_X.transform(X_test)
y1= sc_y.transform(Y_test)
y=y.ravel()
y1=y1.ravel()  


from sklearn.linear_model import LinearRegression

lr = LinearRegression()

lr.fit(X,y)
y_pred_train_lr= lr.predict(X)
y_pred_test_lr= lr.predict(X1)

y_pred_train_lr=pd.DataFrame(y_pred_train_lr)
y_pred_test_lr=pd.DataFrame(y_pred_test_lr)

y1=pd.DataFrame(y1)
y=pd.DataFrame(y)

y_pred_test1_lr= sc_y.inverse_transform (y_pred_test_lr)
y_pred_train1_lr=sc_y.inverse_transform (y_pred_train_lr)

y_test= sc_y.inverse_transform (y1)
y_train= sc_y.inverse_transform (y)

y_pred_test1_rf=pd.DataFrame(y_pred_test1_lr)
y_pred_train1_rf=pd.DataFrame(y_pred_train1_lr)

y_test= pd.DataFrame(y_test)


#summarize the fit of the model on train data
mape_train_lr = mean_absolute_percentage_error(y_train,y_pred_train1_lr)
rmse_train_lr = sqrt(mean_squared_error(y_train,y_pred_train1_lr))
mae_train_lr = metrics.mean_absolute_error(y_train,y_pred_train1_lr)

# summarize the fit of the model on test data
mape_test_lr = mean_absolute_percentage_error(y_test,y_pred_test1_lr)
rmse_test_lr = sqrt(mean_squared_error(y_test,y_pred_test1_lr))
mae_test_lr = metrics.mean_absolute_error(y_test,y_pred_test1_lr)

# train scores
print("The metrics for the training data are: ")
print(mape_train_lr)
print(rmse_train_lr)
print(mae_train_lr)

# test scores
print("The metrics for the testing data are: ")
print(mape_test_lr)
print(rmse_test_lr)
print(mae_test_lr)

lr_time=time.time() - start_time
print("--- %s seconds - Linear Regression ---" % (lr_time))


In [ ]:
# Support Vector Regression
start_time = time.time()

numpy.random.seed(1234)
tf.random.set_seed(1234)

X_train=pd.DataFrame(trainX)
Y_train=pd.DataFrame(trainY)
X_test=pd.DataFrame(testX)
Y_test=pd.DataFrame(testY)

sc_X = StandardScaler()
sc_y = StandardScaler()

X= sc_X.fit_transform(X_train)
y= sc_y.fit_transform(Y_train)
X1= sc_X.transform(X_test)
y1= sc_y.transform(Y_test)
y=y.ravel()
y1=y1.ravel()  

from sklearn.svm import SVR

svr = SVR()

svr.fit(X,y)
y_pred_train_svr= svr.predict(X)
y_pred_test_svr= svr.predict(X1)

y_pred_train_svr=pd.DataFrame(y_pred_train_svr)
y_pred_test_svr=pd.DataFrame(y_pred_test_svr)

y1=pd.DataFrame(y1)
y=pd.DataFrame(y)

y_pred_test1_svr= sc_y.inverse_transform (y_pred_test_svr)
y_pred_train1_svr=sc_y.inverse_transform (y_pred_train_svr)

y_test= sc_y.inverse_transform (y1)
y_train= sc_y.inverse_transform (y)

y_pred_test1_rf=pd.DataFrame(y_pred_test1_svr)
y_pred_train1_rf=pd.DataFrame(y_pred_train1_svr)

y_test= pd.DataFrame(y_test)


#summarize the fit of the model on train data
mape_train_svr = mean_absolute_percentage_error(y_train,y_pred_train1_svr)
rmse_train_svr = sqrt(mean_squared_error(y_train,y_pred_train1_svr))
mae_train_svr = metrics.mean_absolute_error(y_train,y_pred_train1_svr)

# summarize the fit of the model on test data
mape_test_svr = mean_absolute_percentage_error(y_test,y_pred_test1_svr)
rmse_test_svr = sqrt(mean_squared_error(y_test,y_pred_test1_svr))
mae_test_svr = metrics.mean_absolute_error(y_test,y_pred_test1_svr)

# train scores
print("The metrics for the training data are: ")
print(mape_train_svr)
print(rmse_train_svr)
print(mae_train_svr)

# test scores
print("The metrics for the testing data are: ")
print(mape_test_svr)
print(rmse_test_svr)
print(mae_test_svr)

svr_time=time.time() - start_time
print("--- %s seconds - Support Vector Regression ---" % (svr_time))


In [ ]:
# Comparison

In [ ]:
# using whole dataset vs clusters for prediction 
print(mape_test_svr, (mape_test_svr_c1,mape_test_svr_c2,mape_test_svr_c3))
print(rmse_test_svr, (rmse_test_svr_c1,rmse_test_svr_c2,rmse_test_svr_c3))
print(mae_test_svr, (mae_test_svr_c1,mae_test_svr_c2,mae_test_svr_c3))


In [ ]:
# using whole dataset vs clusters for prediction 
print(mape_test_lr, (mape_test_lr_c1,mape_test_lr_c2,mape_test_lr_c3))
print(rmse_test_lr, (rmse_test_lr_c1,rmse_test_lr_c2,rmse_test_lr_c3))
print(mae_test_lr, (mae_test_lr_c1,mae_test_lr_c2,mae_test_lr_c3))


In [ ]:
# using whole dataset vs clusters for prediction 
print(mape_test_svr, (((4/12)*mape_test_svr_c1)+((3/12)*mape_test_svr_c2)+((5/12)*mape_test_svr_c3)))
print(rmse_test_svr, (((4/12)*rmse_test_svr_c1)+((3/12)*rmse_test_svr_c2)+((5/12)*rmse_test_svr_c3)))
print(mae_test_svr, (((4/12)*mae_test_svr_c1)+((3/12)*mae_test_svr_c2)+((5/12)*mae_test_svr_c3)))

In [ ]:
# using whole dataset vs clusters for prediction 
print(mape_test_lr, (((4/12)*mape_test_lr_c1)+((3/12)*mape_test_lr_c2)+((5/12)*mape_test_lr_c3)))
print(rmse_test_lr, (((4/12)*rmse_test_lr_c1)+((3/12)*rmse_test_lr_c2)+((5/12)*rmse_test_lr_c3)))
print(mae_test_lr, (((4/12)*mae_test_lr_c1)+((3/12)*mae_test_lr_c2)+((5/12)*mae_test_lr_c3)))